In [752]:
import random
import time
import base64
import json

# pip install cryptography
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

try:
    from flag import getflag
except Exception:
    def getflag(lv):
        return 'fake{not-the-real-flag}'
    
try:
    from secret import AES_KEYS, AES_TWEAKS
except Exception:
    import secrets
    AES_KEYS = [secrets.token_bytes(64) for _ in range(3)]
    AES_TWEAKS = [secrets.token_bytes(16) for _ in range(3)]


In [760]:
def gen_token():
    ALPHABET='qwertyuiopasdfghjklzxcvbnm1234567890'
    LENGTH=16
    return ''.join([random.choice(ALPHABET) for _ in range(LENGTH)])

def mosaic_filter(s):
    #return s
    if len(s)<=6:
        return '*'*len(s)
    else:
        return s[:4] + '*'*(len(s)-4)
    
def gen_ticket(level, name, stuid, flag):
    if level not in ["1", "2", "3"]:
        return 'Error: 无效的关卡'
    l = int(level) - 1
    
    #print(name, len(name))
    if not 0<len(name)<=[99,22,18][l]:
        return 'Error: 姓名长度不正确'
    if not (len(stuid)==10 and stuid.isdigit()):
        return 'Error: 学号格式不正确'
    
    match l:
        case 0:        
            data = {
                'stuid': stuid,
                'name': name,
                'flag': flag,
                'timestamp': int(time.time()),
            }
        case 1:        
            data = {
                'stuid': stuid,
                'name': name,
                'flag': flag,
                'code': gen_token(),
                'timestamp': int(time.time()),
            }
        case 2:        
            data = {
                'stuid': stuid,
                'code': gen_token(),
                'name': name,
                'flag': flag,
            }
    print(json.dumps(data).encode())
    cipher = Cipher(algorithms.AES(AES_KEYS[l]), modes.XTS(AES_TWEAKS[l])).encryptor()
    ct_bytes = cipher.update(json.dumps(data).encode())
    enc_out = base64.b64encode(ct_bytes).decode()
    
    return enc_out
    
def query_ticket(level, ticket):
    if level not in ["1", "2", "3"]:
        return 'Error: 无效的关卡'
    l = int(level) - 1
    ticket_b64 = ticket.strip()
    if len(ticket_b64) > 1024:
        return 'Error: 太长了'
    
    try:
        ticket = base64.b64decode(ticket_b64)
        cipher = Cipher(algorithms.AES(AES_KEYS[l]), modes.XTS(AES_TWEAKS[l])).decryptor()
        plaintext = cipher.update(ticket)
    except Exception as e:
        print(e)
        return 'Error: 解密购票凭证失败'
    
    print(plaintext)
    print(plaintext.decode('utf-8', 'ignore'))

    try:
        data = json.loads(plaintext.decode('utf-8', 'ignore'))
    except Exception as e:
        print(e)
        return 'Error: 信息解码失败'
    
    return data
    
def flag(level, ticket, code):
    if level not in ["1", "2", "3"]:
        return 'Error: 无效的关卡'
    l = int(level) - 1
    ticket_b64 = ticket.strip()
    code = code
    if len(ticket_b64) > 1024 or len(code) > 1024:
        return 'Error: 太长了'
    
    try:
        ticket = base64.b64decode(ticket_b64)
        cipher = Cipher(algorithms.AES(AES_KEYS[l]), modes.XTS(AES_TWEAKS[l])).decryptor()
        plaintext = cipher.update(ticket)
    except Exception as e:
        print(e)
        return 'Error: 解密购票凭证失败'
    
    try:
        data = json.loads(plaintext.decode('utf-8', 'ignore'))
    except Exception as e:
        print(e)
        return 'Error: 信息解码失败'
        
    if data['flag']!=True:
        return 'Error: 您未选择需要礼品'
        
    if l!=0 and code!=data['code']:
        return 'Error: 兑换码错误'
    
    return '<p>兑换成功，这是你的礼品：</p><br><p>'+getflag(l)+'</p>'

## flag1

In [542]:
len('{"stuid": "1234567890", "name": "11111", "flag":')

48

In [113]:
len('{"stuid": "1234567890", "name": "11111          true            ')

64

In [114]:
len('{"stuid": "1234567890", "name": "111111111111111", "flag": false')

64

In [163]:
s1 = gen_ticket(level="1", name="11111", stuid="1234567890", flag=False)
print(s1)

b'{"stuid": "1234567890", "name": "11111", "flag": false, "timestamp": 1760774539}'
/SROc1TBzvZHwFFp8XCxI//+y/F5bnuaP0P7BW43hKsWXiP7IAPouZ1pJcyZLi03B/C7RyL/rbdIuMXEuVy6MvenCBF8o3CNw37SxnCEWV8=


In [207]:
s2 = gen_ticket(level="1", name="11111          true                 ", stuid="1234567890", flag=False)
print(s2)

b'{"stuid": "1234567890", "name": "11111          true                 ", "flag": false, "timestamp": 1760774743}'
/SROc1TBzvZHwFFp8XCxI//+y/F5bnuaP0P7BW43hKvut/bd31pP+ZXFYScHoB3eR7Kmmt+leZWYCMvLHXIDnnRl6EchRXrCiZ5IK2Fd0GSPTDkoB8V3CdJmIL0eoavUWvW3esAiSrlTO5HkJVGx


In [208]:
s3 = gen_ticket(level="1", name="111111111111111", stuid="1234567890", flag=False)
print(s3)

b'{"stuid": "1234567890", "name": "111111111111111", "flag": false, "timestamp": 1760774743}'
/SROc1TBzvZHwFFp8XCxI//+y/F5bnuaP0P7BW43hKtOsM5cNXHBHLdo0Zf7Lwf+4QV4a9vTKDS2BBvpzVSjBwAslCJKdL5f7WQ24gg0QxXApiZPiCuGnWS4


In [209]:
s4 = base64.b64decode(s1)[:48] + base64.b64decode(s2)[48:64] + base64.b64decode(s3)[64:]
s4 = base64.b64encode(s4)
query_ticket(level="1", ticket=s4)

b'{"stuid": "1234567890", "name": "11111", "flag":true            , "timestamp": 1760774743}'
{"stuid": "1234567890", "name": "11111", "flag":true            , "timestamp": 1760774743}


{'stuid': '1234567890', 'name': '11111', 'flag': True, 'timestamp': 1760774743}

In [124]:
s1 = "XXM5N+IP9SqiPOIkLC2IbaArJZKVzDd7YIXYzVvyIv8ENII+cKhI0/wi9vYIKPt1iifaQjdXtTg3JDlAV7lyEfMNKaN2AnF6DMQPKtNICz4="
s2 = "XXM5N+IP9SqiPOIkLC2IbaArJZKVzDd7YIXYzVvyIv8oBNOtTDfXrhiY6lMJ8zMb259cvocUqw/R+v/uxcSToY/vR4W5136X7u6caMDv1hPpEXqotiJgAoqo/HzXgNL+R9bboIxbv377bQ=="
s3 = "XXM5N+IP9SqiPOIkLC2IbaArJZKVzDd7YIXYzVvyIv/+PB49dQzovCnfA6gzEvetRR7HGI1YLraJuwXHNRoqrtxR1gG7B6cEAFwi+XtTrkpKg5B2+yqS5KkF"

In [125]:
s4 = base64.b64decode(s1)[:48] + base64.b64decode(s2)[48:64] + base64.b64decode(s3)[64:]
s4 = base64.b64encode(s4)
s4

b'XXM5N+IP9SqiPOIkLC2IbaArJZKVzDd7YIXYzVvyIv8ENII+cKhI0/wi9vYIKPt1259cvocUqw/R+v/uxcSTodxR1gG7B6cEAFwi+XtTrkpKg5B2+yqS5KkF'

## flag2

In [244]:
len('e, "code": "wdiu')

16

In [916]:
print(len('{"stuid": "1234567890", "name": "11111", "flag":'))
s1 = gen_ticket(level="2", name="11111", stuid="1234567890", flag=False)
print(s1)

48
b'{"stuid": "1234567890", "name": "11111", "flag": false, "code": "04kfjzkpt0xhenv5", "timestamp": 1761312563}'
T6rKg9MHZJk97Qt4bAEFfnT9UlxPBldJuuJKpEVwLYYV8WwuSUisUXrTj7Vpc6KrwbQLu3KyCs9vuPTVyJLbNfn8Slpsga3RDEU+JR19Z/i+uX2RbNt7jng0rdjHFkSZFaKEu1uq8cHY5Kb/


In [283]:
print(len('{"stuid": "1234567890", "name": "🍕                tru')) # 每个🍕变成12字符，16字节前需要填充15字节
s2 = gen_ticket(level="2", name="🍕                tru", stuid="1234567890", flag=False)
print(s2)

53
b'{"stuid": "1234567890", "name": "\\ud83c\\udf55                tru", "flag": false, "code": "5nh5goo0dnvo9rzb", "timestamp": 1760776675}'
Jydur16yV9DAyrmJhZ+bk8KnOR6I68iUexIeyhLH0YOtTGMfewcV8Sk4hPLouqz/eWEBFy6GTAl43ECZrUzzOwJTaMb7LV731gm/XjjKXApP29iNji4rJrrfSHMzRUVFmqD+6j+l2fu86UCCXLL2O+1thut8ifdwQsT49WjkPOFWkgUANXg=


In [285]:
print(len('{"stuid": "1234567890", "name": "1111111111111111", "flag": fals'))
s3 = gen_ticket(level="2", name="1111111111111111", stuid="1234567890", flag=False)
print(s3)

64
b'{"stuid": "1234567890", "name": "1111111111111111", "flag": false, "code": "hs3lsxty41hqwgvk", "timestamp": 1760776693}'
Jydur16yV9DAyrmJhZ+bk8KnOR6I68iUexIeyhLH0YNJYdE0z91+cXB8uooYmO6EjH9pMusF+ir9lxLjULtfZx4npdLeAUZVJiyWjMDQldEqlxN6iIr+nouVXmlhepdCGQCa3BwlwH293wxLJ3gaYeY79H7rios=


In [286]:
print(len('{"stuid": "1234567890", "name": "1111", "flag": false, "code": "22fblcxgl7guj51m"'))
s4 = gen_ticket(level="2", name="1111", stuid="1234567890", flag=False)
print(s4)

81
b'{"stuid": "1234567890", "name": "1111", "flag": false, "code": "kpetc5i8drobvqi5", "timestamp": 1760776693}'
Jydur16yV9DAyrmJhZ+bk8KnOR6I68iUexIeyhLH0YOjlcj0BRM7ZjoXw3K7BvKDxnSx7Q7zXSEWq5iJw1yZGEpO6Ofzj/IYDBP+EKA88sIj1H4LwIMSkEPH9t/jUc/zCJ6fOjWbyArow4Y=


In [287]:
s5 = base64.b64decode(s1)[:48] + base64.b64decode(s2)[48:64] + base64.b64decode(s3)[64:80] + base64.b64decode(s4)[80:]
s5 = base64.b64encode(s5)
query_ticket(level="2", ticket=s5)

b'{"stuid": "1234567890", "name": "11111", "flag":             true, "code": "hs3l", "timestamp": 1760776693}'
{"stuid": "1234567890", "name": "11111", "flag":             true, "code": "hs3l", "timestamp": 1760776693}


{'stuid': '1234567890',
 'name': '11111',
 'flag': True,
 'code': 'hs3l',
 'timestamp': 1760776693}

In [288]:
s1 = "nqwnvMWhV6vxWdPNzqsUDtep2B/0lF2OJ0GedfOM1KVB9PC3j+pNcjAQwFWQpMG+CEqj4CVmPulTjaxX0bMh7HGPPfhFDFuhAGTLPBadohkE6+RSIboUXrC2oV1fdkE7xOgqo2XEO+HhZtkz"
s2 = "nqwnvMWhV6vxWdPNzqsUDtep2B/0lF2OJ0GedfOM1KUsVVvK4pCIOoeJeam35fQ7jxAGiuA3qY5eQdoQytdIfRkkmSI+mu2Hou/f7ziCZJIofN8aEGCjkyWNycMKY2exSmZVBXP9ReZRjJje9h2sSyHJybX/JF8TVig5kTCZY16hYB73Piw="
s3 = "nqwnvMWhV6vxWdPNzqsUDtep2B/0lF2OJ0GedfOM1KUWXti6bPkdNEBsIMicO0dAc3thIQYGmww4V/PuDqvpKdiafDujshc+7x0u6ewsg14zt76MqkPnDHFJ1GwxCMVIR6ft8hcHf5i7nh/Rk/2K1LplRmKpxcw="
s4 = "nqwnvMWhV6vxWdPNzqsUDtep2B/0lF2OJ0GedfOM1KXRfzO6mHN9RK6Sveyc33iDIRTQRpgRBD+llQxoIHCWwux3TkVYJoivAF+Ogn3t1ztD6VP8VqeJxRIk700W5aNsnveuZxo4kNUMBIs="

In [289]:
s5 = base64.b64decode(s1)[:48] + base64.b64decode(s2)[48:64] + base64.b64decode(s3)[64:80] + base64.b64decode(s4)[80:]
s5 = base64.b64encode(s5)
s5 # code同s3前4位

b'nqwnvMWhV6vxWdPNzqsUDtep2B/0lF2OJ0GedfOM1KVB9PC3j+pNcjAQwFWQpMG+jxAGiuA3qY5eQdoQytdIfdiafDujshc+7x0u6ewsg15D6VP8VqeJxRIk700W5aNsnveuZxo4kNUMBIs='

## flag3

In [921]:
len(json.dumps("𝟘")), len(json.dumps("２")) # 去掉引号，每个全角加5，花体加11

(14, 8)

In [922]:
"𝟘２３１２３１２３１".isdigit()

True

In [923]:
print(len('{"stuid": "１２３4567890", "code": "')) # +15，刚好到48
s1 = gen_ticket(level="3", name="111", stuid="１２３4567890", flag=False)
print(s1)

33
b'{"stuid": "\\uff11\\uff12\\uff134567890", "code": "cn383z64slkayskw", "name": "111", "flag": false}'
/4N5udmtBuHUtE1NpkomWym1arueh42BMVVgU+7nQx1MrM+NuaViZoao7w4VIrwlFxuZmy/dQq5D5PWfnEQwk92lS/QSR40IgbX3iRq0lQV1WgNDQTHo5/hPQ1W+APgP


In [924]:
print(len('{"stuid": "１１𝟘𝟘56789𝟘')) # 3个花体2个全角，刚好补到64，后16位是6789𝟘
s2 = gen_ticket(level="3", name="111", stuid="１１𝟘𝟘56789𝟘", flag=False)
print(s2)

21
b'{"stuid": "\\uff11\\uff11\\ud835\\udfd8\\ud835\\udfd856789\\ud835\\udfd8", "code": "o0slgpbidorjpd10", "name": "111", "flag": false}'
/4N5udmtBuHUtE1NpkomW6cJ7NA+InBPQKAap3uypAIXUdFkFwyztKdDcqISVK1TXeBi8nYFfpRlVyLPPyL//VwMop1GE0K3F4EWRm+DP5kKhklJSUOLwqyJm/WR9c34vGx65lv2EySsJ/VygtYtG4NVv7M/WcxH+NJLrg==


In [925]:
# 密文窃取
print(len('{"stuid": "１１１4567890", "code": "cnab0xevrpx8h4y3')) # 3个全角，补到64
print(len('", "name": "１𝟘--1234𝟘')) # 前四个字符填充到96，name在112后面闭合，中间部分用来偷密文
s4 = gen_ticket(level="3", name="１𝟘--1234𝟘", stuid="１１１4567890", flag=False)
print(s4)

49
21
b'{"stuid": "\\uff11\\uff11\\uff114567890", "code": "5tyadfawo0xq91ua", "name": "\\uff11\\ud835\\udfd8--1234\\ud835\\udfd8", "flag": false}'
/4N5udmtBuHUtE1NpkomW2/2nbjve4NfckrmkioN+ZNMrM+NuaViZoao7w4VIrwliZjxdOv4udvJ3YlXPWWQ6LHnmA+BPmmTHvtffFfII5PbjSQ31ObPYZ2aAoeM7MiRB4dT0qAx0FOETi9ndVj0DB6Ek6k2/AYTL0049JwJxbp7


In [932]:
print(len('{"stuid": "１１１4567890", "code": "cnab0xevrpx8h4y3')) # 3个全角，补到64
print(len('", "name": "12345678901", "flag"')) # 这一段补到96，最后一块是需要偷出来的密文
print(len(': false}'))

tmp = ''.join(random.choices(string.ascii_letters, k=11))
s3 = gen_ticket(level="3", name=tmp, stuid="１１１4567890", flag=False)
print(s3)

49
32
8
b'{"stuid": "\\uff11\\uff11\\uff114567890", "code": "gajrvz9mv6kg6gfu", "name": "OtzSYSuLqXx", "flag": false}'
/4N5udmtBuHUtE1NpkomW2/2nbjve4NfckrmkioN+ZNMrM+NuaViZoao7w4VIrwl1C3w8Hv6rQXWHu42O8vZg+3yHCNISsMn4q8OIfSpxJ6sgqJIP0Jo7IDAyVUPGx3+zXlLwfJN3+g=


In [933]:
s5 = base64.b64decode(s1)[:48] + base64.b64decode(s2)[48:64] + base64.b64decode(s4)[64:96] + base64.b64decode(s3)[80:96] + base64.b64decode(s4)[112:]
s5 = base64.b64encode(s5)
s5 # code为6789𝟘

b'/4N5udmtBuHUtE1NpkomWym1arueh42BMVVgU+7nQx1MrM+NuaViZoao7w4VIrwlXeBi8nYFfpRlVyLPPyL//bHnmA+BPmmTHvtffFfII5PbjSQ31ObPYZ2aAoeM7MiRrIKiSD9CaOyAwMlVDxsd/h6Ek6k2/AYTL0049JwJxbp7'

In [934]:
query_ticket(level="3", ticket=s5)

b'{"stuid": "\\uff11\\uff12\\uff134567890", "code": "6789\\ud835\\udfd8", "name": "\\uff11\\ud835\\udfd8--: false}2\xf6\xaa\xc3\x89\xc8\xaa\x88", "flag": false}'
{"stuid": "\uff11\uff12\uff134567890", "code": "6789\ud835\udfd8", "name": "\uff11\ud835\udfd8--: false}2ÉȪ", "flag": false}


{'stuid': '１２３4567890',
 'code': '6789𝟘',
 'name': '１𝟘--: false}2ÉȪ',
 'flag': False}

## flag3 online

In [882]:
import requests
import re
import string
import html

baseurl = "https://prob14-bilm4vsb.geekgame.pku.edu.cn/"

def my_gen(name, stuid, online=True):
    if online:
        url = baseurl + "3/gen-ticket"
        params = {
            "name": name,
            "stuid": stuid
        }
        response = requests.get(url, params=params)
        pattern = r'：</p><br><p>(.*?)</p>'
        matches = re.findall(pattern, response.text)
        if matches:
            #print(matches[0])
            return matches[0]
        else:
            print(response.text)
    else:
        return gen_ticket(level="3", name=name, stuid=stuid, flag=False)

def my_query(ticket, online=True):
    if online:
        url = baseurl + "3/query-ticket"
        params = {"ticket": ticket}
        response = requests.get(url, params=params)
        pattern = r'姓名：</b> １𝟘--(.*?)</p>'
        matches = re.findall(pattern, response.text)
        if matches:
            return html.unescape(matches[0])
        else:
            return None
    else:
        return query_ticket(level="3", ticket=ticket)

In [856]:
online = True
s1 = my_gen(name="111", stuid="１２３4567890", online=online)
s2 = my_gen(name="111", stuid="１１𝟘𝟘56789𝟘", online=online)
s4 = my_gen(name="１𝟘--1234𝟘", stuid="１１１4567890", online=online)

tmp = ''.join(random.choices(string.ascii_letters, k=11))
s3 = my_gen(name=tmp, stuid="１１１4567890", online=online)

s5 = base64.b64decode(s1)[:48] + base64.b64decode(s2)[48:64] + base64.b64decode(s4)[64:96] + base64.b64decode(s3)[80:96] + base64.b64decode(s4)[112:]
s5 = base64.b64encode(s5).decode() # code为6789𝟘
my_query(ticket=s5, online=online)

': false}K+bO'

In [898]:
for i in range(1000):
    tmp = ''.join(random.choices(string.ascii_letters, k=11))
    s3 = my_gen(name=tmp, stuid="１１１4567890", online=online)

    s5 = base64.b64decode(s1)[:48] + base64.b64decode(s2)[48:64] + base64.b64decode(s4)[64:96] + base64.b64decode(s3)[80:96] + base64.b64decode(s4)[112:]
    s5 = base64.b64encode(s5).decode() # code为6789𝟘
    res = my_query(ticket=s5, online=online)
    if res:
        print(len(res.encode()))
        if len(res.encode()) == 16:
            break

13
13
15
11
14
11
11
11
12
10
10
12
15
13
12
12
11
15
11
14
14
11
11
12
11
13
10
13
11
12
11
12
9
12
12
12
13
11
13
13
13
11
10
13
12
8
10
11
14
12
13
9
12
14
10
13
14
13
11
13
15
12
13
14
15
12
13
11
12
12
12
13
13
10
11
9
15
10
13
13
11
13
10
11
12
12
12
11
14
14
14
13
12
13
12
13
12
13
13
12
13
16


In [900]:
res.encode()

b': false}\xd9\xb78\xd9\x99H-G'

In [903]:
tar = base64.b64decode(s3)[96:104] + res.encode()[8:]
len(tar)

16

In [909]:
print(len('{"stuid": "１１１１１１１890", "code": "cnab0xevrpx8h4y3", "name": "')) # 7个全角，补到96
print(len(':true          }')) # 这一段补到112
print(len(' false}'))
s6 = my_gen(name=":true          }", stuid="１１１１１１１890", online=online)
print(s6)

61
16
7
Utw4jUMDtLHL6i0pdB0QgFoy+sWOrtEfeF4p5gLNdAhZq5rLGtdNxzfQA/AvbuOdwRWqxxO1rj4RBl7cBX7O3lIgJF2hqvaGR4MlhpzlYTq5G0xPklrwNg8SGr2RiWRTf9SDKaG0aOxi+x2z7Jf617oSoVywABGLCPpWCoSZXbfC


In [910]:
s7 = base64.b64decode(s1)[:48] + base64.b64decode(s2)[48:64] + base64.b64decode(s3)[64:80] + tar + base64.b64decode(s6)[96:112]
s7 = base64.b64encode(s7).decode()
s7

'Utw4jUMDtLHL6i0pdB0QgJivB4qTi0ZH6D5lT9oX4HaoXuZtdu97qGwNC7+DeKUUj4FcY4biHv1K5rPK7YfPMmOyU2sTUokXUEFPJpdj73EdoLce/sZF3dm3ONmZSC1Hf9SDKaG0aOxi+x2z7Jf61w=='

In [938]:
# 不会被ignore的字符数量
cnt = 0
for i in range(256):
    c = int.to_bytes(i)
    d = c.decode("utf-8", "ignore")
    if len(d) == 1:
        cnt += 1
cnt

128